### The Nested Clustered Optimization Algorithm

7장의 나머지 부분은 마코위츠의 저주의 근원을 다른 새로운 머신러닝 기반 방법인 중첩 군집 최적화($\text{NCO}$, Nested Clustered Optimization)를 도입하는 데 초점을 맞춘다. $\text{NCO}$는 ‘래퍼(wrapper)’라고 알려진 알고리즘의 종류에 속한다. 어떤 효율적 경계의 구성 원소가 계산되는지 여부나 어떤 제약 조건들이 부과되는지 여부에 대해서는 무관하다. $\text{NCO}$는 마코위츠의 저주가 기존의 평균–분산 배분 방법에 미치는 영향을 해결하기 위한 전략을 제공한다.

#### 1. Correlation Clustering

우선 $\text{NCO}$ 알고리즘의 첫 번째 단계는 상관 행렬을 군집화하는 것이다. 이 작업에는 최적의 군집 수를 찾는 작업이 포함된다. 한 가지 가능성은 $\text{ONC}$ 알고리즘(Chapter 4)을 적용하는 것이지만, $\text{NCO}$는 군집 수를 결정하는 데 사용되는 어떤 알고리즘이 사용되는가에 대해서는 무관하다. $T/N$이 상대적으로 낮은 큰 행렬의 경우 2장에서 설명한 방법에 따라 군집화 전에 상관 행렬의 잡음을 제거하는 것이 바람직하다. 아래 코드는 이 절차를 구현한다. Chapter 2에서 소개한 `deNoiseCov` 함수를 사용해 잡음이 제거된 공분산 행렬(cov1)을 계산한다. 주의 사항으로 인수 $q$는 관측 행렬의 행 수와 열 수 사이의 비율을 알려 준다. `bWidth = 0`일 때 공분산 행렬의 잡음은 제거되지 않는다. `cov2corr` 함수를 사용해 결과 공분산 행렬을 상관 행렬로 표준화한다. 그런 다음 Chapter 4에서 소개한 `clusterKMeansBase` 함수를 사용해 정제된 상관 행렬을 군집화한다. `maxNumClusters` 인수는 상관 행렬의 열 수의 절반으로 설정된다. 그 이유는 단일 항목 군집은 행렬의 조건 수 증가를 유발하지 않기 때문에 최소 크기가 2인 군집만 고려하면 되기 때문이다. 군집 수가 더 작을 것으로 예상되는 경우 더 낮은 `maxNumClusters`를 사용해 계산을 가속화할 수 있다.


일반적인 질문은 `corr1` 또는 `corr1.abs()`를 군집화해야 하는가다. 모든 상관관계가 음수가 아닌 경우 `corr1`과 `corr1.abs()` 군집화는 동일한 결과를 산출한다. 일부 상관관계가 음수일 경우 답은 더 복잡하며, 관측된 입력의 수치 특성에 따라 달라진다. 두 가지를 모두 시도해 보고, 몬테카를로 실험에서 어떤 군집화가 특정 `corr1`에 더 잘 작용하는지 볼 것을 추천한다.

#### 2. Intracluster Weights

NCO 알고리즘의 두 번째 단계는 잡음이 제거된 공분산 행렬, `cov1`을 사용해 최적의 군집 내 배분을 계산하는 것이다. 아래 코드는 이 절차를 구현한다. 단순성을 위해 `minVarPort` 함수에 구현된 최소 분산 배분을 기본으로 설정했다. 그러나 이 절차는 다른 대체적인 배분 방법의 사용도 허용한다. 측정된 군집 내 가중치를 사용해 군집 간 상관관계를 나타내는 축약된 공분산 행렬인 `cov2`를 도출할 수 있다.

#### 3. Intercluster Weights

NCO 알고리즘의 세 번째 단계는 축약된 공분산 행렬인 cov2를 사용해 최적의 군집 간 배분을 계산하는 것이다. 구상의 이 공분산 행렬은 대각 행렬에 가깝고, 최적화 문제는 이상적인 마코위츠 사례에 가깝다. 즉 군집화와 군집 내 최적화 단계를 통해 ‘마코위츠–저주(Markowitz-cursed)’ 문제($|\rho| > 0$)를 올바른 작동하는 문제($\rho \sim 0$)으로 전환할 수 있다.

아래 코드는 이 절차를 구현한다. 군집 내 배분 단계의 경우 `minVarPort` 함수에서 사용된 것과 동일한 배분 절차를 적용한다. 증권별 최종 배분은 `WAll0` 데이터 프레임에 의해 표현되며, 이는 군집 내 비중과 군집 간 비중을 곱한 결과로 나타난다.